In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 0.4 extraction within the range

In [ ]:
import pandas as pd
import os

# Path setting
directory_path = '/content/drive/MyDrive/Kwargs/esg관련도/test'

Initialize the data frame to label to # 0
selected_data = pd.DataFrame()

# Process all CSV files
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)

# Select data that meets specific conditions
        filtered_df = df[(df['sentiment_score'] >= -0.4) & (df['sentiment_score'] <= 0.4)]



# Set to Label 0
        filtered_df['label'] = 0

# Leave only the required columns, and change the name to text to text
        filtered_df = filtered_df[['full_text', 'label']].rename(columns={'full_text': 'text'})

# Add to selected data frame
        selected_data = pd.concat([selected_data, filtered_df], ignore_index=True)


selected_data.sample()


<ipython-input-6-f667d910dea6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['label'] = 0
<ipython-input-6-f667d910dea6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['label'] = 0
<ipython-input-6-f667d910dea6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

text  label
954  한진 때와 판박이불 붙는 금호석화 표대결 머니투데이 김성은 기자 월 주주총회를 앞두...      0

### Extraction of irrelevant data

In [ ]:
import os
import json
import pandas as pd
from tqdm import tqdm

# List of file path and file name
base_path = '/content/drive/MyDrive/Kwargs/무관데이터/kakao_1'  # Colab에서는 '/content/'를 사용. 필요 시 수정
save_path =  '/content/drive/MyDrive/Kwargs/무관데이터'
file_prefix = 'KAKAO_'
file_suffix = '.json'

# List for storing data
data = []

# Setting file range
prefix_range_start = 899
prefix_range_end = 1798
suffix_range_start = 1
suffix_range_end = 14

# Display progress using TQDM
total_files = (prefix_range_end - prefix_range_start + 1) * (suffix_range_end - suffix_range_start + 1)
progress_bar = tqdm(total=total_files, desc="Processing files")

# Data extraction by traveling each file
for prefix in range(prefix_range_start, prefix_range_end + 1):
    for suffix in range(suffix_range_start, suffix_range_end + 1):
        file_name = f"{file_prefix}{prefix}_{str(suffix).zfill(2)}{file_suffix}"
        file_path = os.path.join(base_path, file_name)

        if not os.path.exists(file_path):
            print(f"파일을 찾을 수 없습니다: {file_path}")
            progress_bar.update(1)
            continue

        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = json.load(file)
                if 'info' in content:
                    for item in content['info']:
                        norm_texts = [line['norm_text'] for line in item['annotations']['lines']]
                        combined_text = ' '.join(norm_texts)
                        data.append({'filename': file_name, 'norm_text': combined_text})
        except json.JSONDecodeError:
            print(f"JSONDecodeError: 파일을 읽는 중 오류가 발생했습니다: {file_path}")
        except Exception as e:
            print(f"예상치 못한 오류가 발생했습니다: {file_path}, 오류: {str(e)}")

# Update progress
        progress_bar.update(1)

#Data frame creation
df = pd.DataFrame(data)

#Save as CSV
output_file = os.path.join(save_path, 'combined_norm_texts.csv')
df.to_csv(output_file, index=False, encoding='utf-8-sig')

# TQDM exit
progress_bar.close()

print(f"CSV 파일로 저장되었습니다: {output_file}")



Processing files:   0%|          | 63/12600 [01:43<5:43:14,  1.64s/it]

Processing files:   1%|▏         | 159/12600 [00:50<1:51:56,  1.85it/s]

KeyboardInterrupt: 

### Negative Conversion Text

In [ ]:
import pandas as pd

def extract_and_save_sentences(input_file, output_file):
    sentences = []
    with open(input_file, 'r', encoding='utf-8') as file:
        for line in file:
            if line.startswith("Back to Korean:"):
                sentence = line.split("Back to Korean:")[1].strip()
                sentences.append(sentence)

    df = pd.DataFrame(sentences, columns=["Back to Korean Sentences"])
    df.to_csv(output_file, index=False, encoding='utf-8')

# Usage
input_file = '/content/drive/MyDrive/Colab Notebooks/부정 텍스트 변환 0710.txt'
output_file = '/content/drive/MyDrive/Kwargs/부정 텍스트 변환 0710.csv'
extract_and_save_sentences(input_file, output_file)


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Kwargs/부정 텍스트 변환 0710.csv')
df

Back to Korean Sentences
0     조직 내부의 ESG위원회를 설치하거나 외부 전문가를 참여시켜 주주의 의사에 부합하고...
1     ESG 관점에서 모듈형 건설 방식은 혁신적이고 지속 가능하지만 공장에서 현장까지 건...
2     ESG 관점에서 베트남(비나)은 2021년 평가에서 긍정적인 성과를 거두기 위해 인...
3     근로 기본권을 지키지 아니한다 □ 지방법에 따라 근로자의 근로 기본권을 존중하지 못한다.
4     ESG 관점에서 사외이사는 SEC 규정 4200이 허용되지 않는 한 "현 회계연도 ...
...                                                 ...
5116  ESG 관점에서 3단계 전략적 이슈 선택은 다양한 성과 이슈를 효과적으로 다루지 못...
5117  나아가, 우리는 ESG 관점에서 스범위 3 배출량을 측정하고 완화하기 위해 공급 사...
5118  ESG 관점에서 부패에 대한 관용을 허용하지 않고 책임 집행과 관련하여 개인 이익 ...
5119  ESG 관점에서 SK스퀘어와의 분할은 비핵심 ICT 사업 역량을 소홀히 하고 사업 ...
5120  SK 지오센트럼은 ESG 관점에서 울산 현장에서 20만톤의 폐플라스틱을 재활용할 수...

[5121 rows x 1 columns]

### 3 species

In [ ]:
import pandas as pd
import os
from tqdm import tqdm

# File path
#base_path_label_0 = '/content/drive/MyDrive/Kwargs/무관데이터/combined_norm_texts.csv'
df_label_0 = selected_data
base_path_label_1 = '/content/drive/MyDrive/Kwargs/esg보고서/라벨링/'
#base_path_label_minus_1 = '/content/drive/MyDrive/Kwargs/부정 텍스트 변환 0710.csv'
base_path_label_minus_1 = '/content/drive/MyDrive/Kwargs/Generated_Combinations.csv'

# Import the file list
files_label_1 = [os.path.join(base_path_label_1, f) for f in os.listdir(base_path_label_1) if f.endswith('.csv')]

In [ ]:
#Label 0 Data Reading
df_label_0 = pd.read_csv(base_path_label_0)
df_label_0['label'] = 0
df_label_0.rename(columns={'norm_text': 'text'}, inplace=True)

In [ ]:
# Read Label 1 Data
data_label_1 = []
for file in tqdm(files_label_1, desc="Processing label 1 files"):
    df = pd.read_csv(file)
    df['label'] = 1
    df.rename(columns={'sentence': 'text'}, inplace=True)
    data_label_1.append(df)
df_label_1 = pd.concat(data_label_1, ignore_index=True)

Processing label 1 files: 100%|██████████| 150/150 [00:05<00:00, 28.13it/s]


In [ ]:
#Label -1 Data Reading
df_label_minus_1 = pd.read_csv(base_path_label_minus_1)
df_label_minus_1['label'] = -1
df_label_minus_1.rename(columns={'norm_text': 'text'}, inplace=True)
df_label_minus_1.rename(columns={'Back to Korean Sentences': 'text'}, inplace=True)

In [ ]:
df_label_1.rename(columns={'문장': 'text'}, inplace=True)

### Data imbalance resolution

In [ ]:
import random
from eda import eda
import pandas as pd

# Import data
data = df_label_0

# Select data from classes with few labels
minority_class = data[data['label'] == data['label'].value_counts().idxmin()]

# Number of data to be enhanced
augmentation_factor = 1.7
num_augment = int(len(minority_class) * (augmentation_factor - 1))

augmented_texts = []
augmented_labels = []

for i in range(num_augment):
    original_text = random.choice(minority_class['text'].values)
    augmented_text = eda(original_text, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=1)
    augmented_texts.append(augmented_text[0])
    augmented_labels.append(minority_class['label'].values[0])

# Enhancement data frame creation
augmented_data = pd.DataFrame({'text': augmented_texts, 'label': augmented_labels})

# Added augmented data to the original data
balanced_data = pd.concat([data, augmented_data], ignore_index=True)

#Save the results as CSV
df_label_0 = balanced_data

ModuleNotFoundError: No module named 'eda'

### Data enhancement

In [ ]:
# Increased data with 0 label 0
label_0_data = df_label_0[df_label_0['label'] == 0]
df_label_0 = pd.concat([df_label_0, label_0_data, label_0_data], ignore_index=True)

### Start to join in earnest

In [ ]:
import pandas as pd

# Data continuous function
def combine_texts(df, num_texts):
    combined_texts = []
    df['text'].fillna('', inplace=True)  # NaN 값을 빈 문자열로 대체
    for i in range(0, len(df), num_texts):
        combined_text = ' '.join(df['text'][i:i+num_texts])
        combined_texts.append(combined_text)
    return pd.DataFrame(combined_texts, columns=['text'])

In [ ]:
# Label 0: 3 pieces
df_label_0_combined = combine_texts(df_label_0, 1)
df_label_0_combined['label'] = 0

#Label 1: 15 pieces
df_label_1_combined = combine_texts(df_label_1, 15)
df_label_1_combined['label'] = 1

#Label -1: used as it is
df_label_m1_combined = df_label_minus_1[['text', 'label']]

In [ ]:

# All data integration
df_combined = pd.concat([df_label_0_combined, df_label_1_combined, df_label_m1_combined], ignore_index=True)

# Replace the NAN value with an empty string
df_combined['text'].fillna('', inplace=True)

# Calculation of the average length and number of text rows by label
df_combined['text_length'] = df_combined['text'].apply(len)
label_stats = df_combined.groupby('label').agg({'text_length': 'mean', 'text': 'count'}).rename(columns={'text_length': 'avg_text_length', 'text': 'count'})

#Integrated data storage
output_file = '/content/drive/MyDrive/Kwargs/combined_labeled_data_balanced_0716.csv'
df_combined.to_csv(output_file, index=False, encoding='utf-8-sig')

# Output
print(f"CSV 파일로 저장되었습니다: {output_file}")
print(label_stats)


CSV 파일로 저장되었습니다: /content/drive/MyDrive/Kwargs/combined_labeled_data_balanced_0716.csv
       avg_text_length  count
label                        
-1         2145.809000   5000
 0         1466.760204   5292
 1         2288.836983  19360


In [ ]:
# Undersampling for the smallest data for each label
min_count = df_combined['label'].value_counts().min()


df_label_0_final = df_label_0_combined.sample(n=min_count, random_state=42)
df_label_1_final = df_label_1_combined.sample(n=min_count, random_state=42)
df_label_minus_1_final = df_label_m1_combined.sample(n=min_count, random_state=42)

# Final data integration
df_final = pd.concat([df_label_0_final, df_label_1_final, df_label_minus_1_final], ignore_index=True)

# Calculation of the average length and number of text rows by label
df_final['text_length'] = df_final['text'].apply(len)
label_stats_final = df_final.groupby('label').agg({'text_length': 'mean', 'text': 'count'}).rename(columns={'text_length': 'avg_text_length', 'text': 'count'})

#Integrated data storage
output_file_final = '/content/drive/MyDrive/Kwargs/combined_labeled_data_balanced_0719.csv'
df_final.to_csv(output_file_final, index=False, encoding='utf-8-sig')

# Output
print(f"CSV 파일로 저장되었습니다: {output_file_final}")
print(label_stats_final)

CSV 파일로 저장되었습니다: /content/drive/MyDrive/Kwargs/combined_labeled_data_balanced_0719.csv
       avg_text_length  count
label                        
-1           2145.8090   5000
 0           1467.5248   5000
 1           2315.9126   5000
